In [0]:
import zipfile
import pandas as pd

# Step 1: Load dataset from Volumes
zip_path = "/Volumes/workspace/default/netflix/netflix.csv.zip"
extract_path = "/Volumes/workspace/default/netflix/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

csv_path = extract_path + "netflix_titles.csv"
df = pd.read_csv(csv_path)

# Display full dataset (or a large subset)
pd.set_option('display.max_rows', None)    # temporarily allow showing all rows
display(df.head(10))
pd.reset_option('display.max_rows')

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,null,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
s2,TV Show,Blood & Water,null,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",null,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
s4,TV Show,Jailbirds New Orleans,null,null,null,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
s5,TV Show,Kota Factory,null,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."
s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, Henry Thomas, Kristin Lehman, Samantha Sloyan, Igby Rigney, Rahul Kohli, Annarah Cymone, Annabeth Gish, Alex Essoe, Rahul Abburi, Matt Biedel, Michael Trucco, Crystal Balint, Louis Oliver",null,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries","The arrival of a charismatic young priest brings glorious miracles, ominous mysteries and renewed religious fervor to a dying town desperate to believe."
s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, Sofia Carson, Liza Koshy, Ken Jeong, Elizabeth Perkins, Jane Krakowski, Michael McKean, Phil LaMarr",null,"September 24, 2021",2021,PG,91 min,Children & Family Movies,"Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegasi and Unicorns should be pals — and, hoof to heart, she’s determined to prove it."
s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri","United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past."
s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Hollywood",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV","A talented batch of amateur bakers face off in a 10-week competition, whipping up their best dishes in the hopes of being named the U.K.'s best."
s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, Timothy Olyphant, Daveed Diggs, Skyler Gisondo, Laura Harrier, Rosalind Chao, 

In [0]:
# Step 2: Inspect the dataset
print("Shape:", df.shape)
print("\nMissing values per column:\n", df.isnull().sum())
df.info()


Shape: (8807, 12)

Missing values per column:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   

In [0]:
# Simple fix: ensure object / string columns are plain strings
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str)

# Now convert to Spark
spark_df = spark.createDataFrame(df)


In [0]:
# Step 3: Remove duplicates & fill missing values safely

# Remove duplicate rows
df = df.drop_duplicates()

# Fill missing values for categorical/text columns
df['director'] = df['director'].fillna("Unknown")
df['cast'] = df['cast'].fillna("Unknown")
df['country'] = df['country'].fillna("Unknown")
df['rating'] = df['rating'].fillna("Not Rated")
df['date_added'] = df['date_added'].fillna("Unknown")
df['listed_in'] = df['listed_in'].fillna("Unknown")
df['duration'] = df['duration'].fillna("Unknown")
df['description'] = df['description'].fillna("No description available")

In [0]:
# Step 4: Handle missing values
df['director'] = df['director'].fillna("Unknown")
df['cast'] = df['cast'].fillna("Unknown")
df['country'] = df['country'].fillna("Unknown")
df['rating'] = df['rating'].fillna("Not Rated")
df['date_added'] = df['date_added'].fillna("Unknown")

print("✅ Missing values handled")
print(df.isnull().sum())


✅ Missing values handled
show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


In [0]:
# Step 5: Normalize categorical text
df['type'] = df['type'].str.strip()
df['rating'] = df['rating'].str.strip()
df['country'] = df['country'].str.strip()

print("Unique values in 'type':", df['type'].unique())
print("Unique values in 'rating':", df['rating'].unique())


Unique values in 'type': ['Movie' 'TV Show']
Unique values in 'rating': ['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' '74 min' '84 min' '66 min' 'NR' 'nan' 'TV-Y7-FV' 'UR']


In [0]:
# 7.1: Group ratings into broader categories
rating_map = {
    "TV-MA": "Adult", "R": "Adult", "NC-17": "Adult",
    "PG-13": "Teens", "TV-14": "Teens",
    "G": "Kids", "PG": "Kids", "TV-Y": "Kids", "TV-Y7": "Kids",
    "TV-PG": "Kids"
}
df['rating_group'] = df['rating'].map(rating_map).fillna("Other")

print("Unique rating groups:", df['rating_group'].unique())
print("Counts per group:\n", df['rating_group'].value_counts())


Unique rating groups: ['Teens' 'Adult' 'Kids' 'Other']
Counts per group:
 Adult    4009
Teens    2650
Kids     1832
Other     316
Name: rating_group, dtype: int64


In [0]:
# 7.2: Extract primary genre (first genre in the list)
df['primary_genre'] = df['listed_in'].apply(
    lambda x: x.split(',')[0].strip() if pd.notnull(x) and isinstance(x, str) else "Unknown"
)

print("Unique primary genres:", df['primary_genre'].unique()[:10])
print(df['primary_genre'].value_counts().head(10))


Unique primary genres: ['Documentaries' 'International TV Shows' 'Crime TV Shows' 'Docuseries'
 'TV Dramas' 'Children & Family Movies' 'Dramas' 'British TV Shows'
 'Comedies' 'TV Comedies']
Dramas                      1600
Comedies                    1210
Action & Adventure           859
Documentaries                829
International TV Shows       774
Children & Family Movies     605
Crime TV Shows               399
Kids' TV                     388
Stand-Up Comedy              334
Horror Movies                275
Name: primary_genre, dtype: int64


In [0]:
# 7.3: Label encode the primary_genre
df['primary_genre_code'] = df['primary_genre'].astype('category').cat.codes

# Show mapping
genre_categories = df['primary_genre'].astype('category').cat.categories
print("Genre categories:", list(genre_categories))
print("Codes used:", df['primary_genre_code'].unique())


Genre categories: ['Action & Adventure', 'Anime Features', 'Anime Series', 'British TV Shows', 'Children & Family Movies', 'Classic & Cult TV', 'Classic Movies', 'Comedies', 'Crime TV Shows', 'Cult Movies', 'Documentaries', 'Docuseries', 'Dramas', 'Horror Movies', 'Independent Movies', 'International Movies', 'International TV Shows', "Kids' TV", 'LGBTQ Movies', 'Movies', 'Music & Musicals', 'Reality TV', 'Romantic Movies', 'Romantic TV Shows', 'Sci-Fi & Fantasy', 'Spanish-Language TV Shows', 'Sports Movies', 'Stand-Up Comedy', 'Stand-Up Comedy & Talk Shows', 'TV Action & Adventure', 'TV Comedies', 'TV Dramas', 'TV Horror', 'TV Sci-Fi & Fantasy', 'TV Shows', 'Thrillers']
Codes used: [10 16  8 11 31  4 12  3  7 30 35 13 17  0 21  2 15 24  6 34 27 29 19 28
  5  1 23  9 14 32 25 20 22 18 33 26]


In [0]:
## 7.5: Frequency encode country (proportion of dataset for each country)
country_freq = df['country'].value_counts(normalize=True)
df['country_freq'] = df['country'].map(country_freq)

print("Some country frequencies:\n", df[['country', 'country_freq']].drop_duplicates().head(10))


Some country frequencies:
                                               country  country_freq
0                                       United States      0.319973
1                                        South Africa      0.003406
2                                                 nan      0.094357
4                                               India      0.110367
7   United States, Ghana, Burkina Faso, United Kin...      0.000114
8                                      United Kingdom      0.047576
12                            Germany, Czech Republic      0.000227
17                                             Mexico      0.012490
21                                             Turkey      0.011922
25                                          Australia      0.009879


In [0]:
import pandas as pd
import zipfile

zip_path = "/Volumes/workspace/default/netflix/netflix.csv.zip"
extract_path = "/Volumes/workspace/default/netflix/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

csv_path = extract_path + "netflix_titles.csv"
df = pd.read_csv(csv_path)





In [0]:
rating_map = {
    "TV-MA": "Adult", "R": "Adult", "NC-17": "Adult",
    "PG-13": "Teens", "TV-14": "Teens",
    "G": "Kids", "PG": "Kids", "TV-Y": "Kids", "TV-Y7": "Kids",
    "TV-PG": "Kids"
}
df['rating_group'] = df['rating'].map(rating_map).fillna("Other")


In [0]:
df['primary_genre'] = df['listed_in'].apply(
    lambda x: x.split(',')[0].strip() if isinstance(x, str) else "Unknown")

In [0]:
df['primary_genre_code'] = df['primary_genre'].astype('category').cat.codes



In [0]:
type_dummies = pd.get_dummies(df['type'], prefix='type')
df = pd.concat([df, type_dummies], axis=1)


In [0]:

country_freq = df['country'].value_counts(normalize=True)
df['country_freq'] = df['country'].map(country_freq)
def categorize_duration(val):
    if pd.isnull(val) or not isinstance(val, str):
        return "Unknown"
    if "Season" in val:
        return "TV Show"
    try:
        minutes = int(val.split()[0])
        if minutes < 60:
            return "Short"
        elif minutes <= 120:
            return "Standard"
        else:
            return "Long"
    except:
        return "Unknown"

df['duration_category'] = df['duration'].apply(categorize_duration)

print("Country freq and duration categories created")






Country freq and duration categories created


In [0]:
## Cast int8 and int16 columns to int32 to avoid Arrow type errors
for col in df.select_dtypes(include=['int8', 'int16']).columns:
    df[col] = df[col].astype('int32')

print("Final clean dataset shape:", df.shape)
print("\nMissing values after cleaning:\n", df.isnull().sum())

# Convert to Spark DataFrame before display
spark_df = spark.createDataFrame(df)


Final clean dataset shape: (8807, 19)

Missing values after cleaning:
 show_id                  0
type                     0
title                    0
director              2634
cast                   825
country                831
date_added              10
release_year             0
rating                   4
duration                 3
listed_in                0
description              0
rating_group             0
primary_genre            0
primary_genre_code       0
type_Movie               0
type_TV Show             0
country_freq           831
duration_category        0
dtype: int64


In [0]:
cleaned_path = "/Volumes/workspace/default/netflix/netflix_titles_clean.csv"
df.to_csv(cleaned_path, index=False)
print("Saved cleaned CSV at:", cleaned_path)


Saved cleaned CSV at: /Volumes/workspace/default/netflix/netflix_titles_clean.csv
